# DECISION TREE Y RANDOM FOREST
---

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

In [35]:
df_est = pd.read_csv("datos/05-bikes_encoding_est.csv", index_col = 0)
df_est.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,0,0,1,0,0,1,-0.691698,1.254842,-0.360109,654
1,0,0,1,0,2,1,1,-0.752316,0.465708,0.867064,670
2,0,0,1,0,2,1,2,-1.759976,-1.395307,0.863860,1229
3,1,0,1,0,0,0,2,-1.620681,-0.293988,-0.362199,1454
4,1,0,2,0,0,1,2,-1.515533,-1.397579,0.008406,1518


In [36]:
df_est.season.unique()

array([0, 1, 2], dtype=int64)

In [37]:
df_est.isnull().sum()

season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
atemp         0
hum           0
windspeed     0
registered    0
dtype: int64

In [38]:
# Creamos una clase para poder hacer el decision tree y el random forest más rápido
class Regres_lineal:
    def __init__(self, dataf, col_pred):
        
        self.dataf = dataf
        self.col_pred = col_pred

    def tree_param(self):
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        arbol = DecisionTreeRegressor(random_state =0)
    
        arbol.fit(x_train, y_train)

        max_feat = np.sqrt(len(x_train.columns))
        max_dep = arbol.tree_.max_depth
        return(max_feat, max_dep)

    def metricas(self, y_te, y_tr, y_te_pred, y_tr_pred, tipo_modelo):
        resultados = {'MAE': [mean_absolute_error(y_te, y_te_pred), mean_absolute_error(y_tr, y_tr_pred)],
                    'MSE': [mean_squared_error(y_te, y_te_pred), mean_squared_error(y_tr, y_tr_pred)],
                    'RMSE': [np.sqrt(mean_squared_error(y_te, y_te_pred)), np.sqrt(mean_squared_error(y_tr, y_tr_pred))],
                    'R2':  [r2_score(y_te, y_te_pred), r2_score(y_tr, y_tr_pred)],
                    "set": ["test", "train"]}
        df = pd.DataFrame(resultados)
        df["modelo"] = tipo_modelo
        return df

    def grid_s_decision(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= GradientBoostingRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

    def grid_s_forest(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= RandomForestRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

In [39]:

rg_est = Regres_lineal(df_est, "registered")

In [40]:
features_est, depth_est = rg_est.tree_param()

In [41]:
features_est #El número máx será 3

3.1622776601683795

In [42]:
depth_est # La profundidad máxima será 22

22

In [43]:
param1 = {"max_depth": [16, 18, 20], 
        "max_features": [3,4],
        "min_samples_split": [50, 100],
        "min_samples_leaf": [50,100]} 

In [44]:
best_model1, result1 = rg_est.grid_s_decision(param1, "GradientBoosting Estand I")

In [45]:
best_model1

GradientBoostingRegressor(max_depth=16, max_features=3, min_samples_leaf=50,
                          min_samples_split=100)

In [46]:
result1

,MAE,MSE,RMSE,R2,set,modelo
0,563.374468,508228.199755,712.901255,0.786520,test,GradientBoosting Estand I
1,486.777676,397037.058521,630.108767,0.836592,train,GradientBoosting Estand I


In [47]:
param2 = {"max_depth": [4,6,8,10, 12, 14, 16, 18, 20], 
        "max_features": [2, 3,4],
        "min_samples_split": [10, 20, 40, 50, 60],
        "min_samples_leaf": [10, 20, 40, 50, 60]} 

In [48]:
best_model2, result2 = rg_est.grid_s_decision(param2, "GradientBoosting Estand II")

In [49]:
best_model2

GradientBoostingRegressor(max_depth=4, max_features=3, min_samples_leaf=10,
                          min_samples_split=20)

In [50]:
result2

,MAE,MSE,RMSE,R2,set,modelo
0,538.246738,478829.524885,691.975090,0.798869,test,GradientBoosting Estand II
1,399.940337,273385.932929,522.863207,0.887483,train,GradientBoosting Estand II


In [51]:
param3 = {"max_depth": [6, 8, 10, 12, 14, 16, 18], 
        "max_features": [3,4],
        "min_samples_split": [10, 20, 40],
        "min_samples_leaf": [10, 20, 40]} 

In [52]:
best_model3, result3 = rg_est.grid_s_forest(param3, "RandForest Estand I")

In [53]:
best_model3

RandomForestRegressor(max_depth=10, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [54]:
result3

,MAE,MSE,RMSE,R2,set,modelo
0,628.482234,605666.105987,778.245531,0.745592,test,RandForest Estand I
1,573.333874,521076.264899,721.856125,0.785542,train,RandForest Estand I


In [55]:
param4 = {"max_depth": [12, 14, 16, 18], 
        "max_features": [4,5],
        "min_samples_split": [10, 20, 40, 50],
        "min_samples_leaf": [10, 20, 40, 50]} 

In [56]:
best_model4, result4 = rg_est.grid_s_forest(param4, "RandForest Estand II")

In [57]:
best_model4

RandomForestRegressor(max_depth=18, max_features=5, min_samples_leaf=10,
                      min_samples_split=10)

In [58]:
result4

,MAE,MSE,RMSE,R2,set,modelo
0,601.243639,571601.442752,756.043281,0.759900,test,RandForest Estand II
1,554.247909,498162.979288,705.806616,0.794972,train,RandForest Estand II


In [59]:
df_sinest = pd.read_csv("datos/04-bikes_encoding_sinest.csv", index_col = 0)
df_sinest.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,0,0,1,0,0,1,18.18125,80.5833,10.749882,654
1,0,0,1,0,2,1,1,17.68695,69.6087,16.652113,670
2,0,0,1,0,2,1,2,9.47025,43.7273,16.636703,1229
3,1,0,1,0,0,0,2,10.60610,59.0435,10.739832,1454
4,1,0,2,0,0,1,2,11.46350,43.6957,12.522300,1518


In [60]:
rg_sinest = Regres_lineal(df_sinest, "registered")

In [61]:
features_sinest, depth_sinest = rg_sinest.tree_param()

In [62]:
features_sinest #El número máx será 3

3.1622776601683795

In [63]:
depth_sinest # La profundidad máxima será 22

22

In [64]:
best_model5, result5 = rg_sinest.grid_s_decision(param1, "GradientBoosting SinEstand I")

In [65]:
result5

,MAE,MSE,RMSE,R2,set,modelo
0,568.142311,516260.958168,718.513019,0.783146,test,GradientBoosting SinEstand I
1,480.692055,380784.591493,617.077460,0.843281,train,GradientBoosting SinEstand I


In [66]:
best_model6, result6 = rg_sinest.grid_s_decision(param2, "GradientBoosting SinEstand II")

In [67]:
result6

,MAE,MSE,RMSE,R2,set,modelo
0,568.620294,526373.351751,725.515921,0.778898,test,GradientBoosting SinEstand II
1,445.231652,332430.095874,576.567512,0.863182,train,GradientBoosting SinEstand II


In [68]:
best_model7, result7 = rg_sinest.grid_s_forest(param3, "RandForest SinEstand I")

In [69]:
result7

,MAE,MSE,RMSE,R2,set,modelo
0,631.242906,620694.510690,787.841679,0.739279,test,RandForest SinEstand I
1,569.549481,517847.635406,719.616311,0.786871,train,RandForest SinEstand I


In [70]:
best_model8, result8 = rg_sinest.grid_s_forest(param4, "RandForest SinEstand II")

In [71]:
result8

,MAE,MSE,RMSE,R2,set,modelo
0,590.248378,558889.455053,747.589095,0.765240,test,RandForest SinEstand II
1,551.312072,499689.552415,706.887228,0.794344,train,RandForest SinEstand II


In [72]:
param5 = {"max_depth": [22], 
        "max_features": [4],
        "min_samples_split": [8],
        "min_samples_leaf": [8]}

In [73]:
best_model9, result9 = rg_sinest.grid_s_forest(param5, "RandForest SinEstand III")

In [74]:
result9

,MAE,MSE,RMSE,R2,set,modelo
0,604.399417,568885.259718,754.244827,0.761041,test,RandForest SinEstand III
1,539.596209,470394.079122,685.852812,0.806401,train,RandForest SinEstand III


In [75]:
best_model10, result10 = rg_est.grid_s_forest(param5, "RandForest Estand III")

In [76]:
result10

,MAE,MSE,RMSE,R2,set,modelo
0,608.095259,576590.258361,759.335406,0.757805,test,RandForest Estand III
1,544.291209,473926.185539,688.422970,0.804947,train,RandForest Estand III


In [77]:
param6 = {"max_depth": [18,20], 
        "max_features": [3,4],
        "min_samples_split": [8, 15],
        "min_samples_leaf": [8, 15]}

In [78]:
best_model11, result11 = rg_est.grid_s_forest(param6, "RandForest Estand IV")

In [79]:
best_model11

RandomForestRegressor(max_depth=18, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [80]:
result11

,MAE,MSE,RMSE,R2,set,modelo
0,599.896683,551262.221084,742.470350,0.768444,test,RandForest Estand IV
1,541.062731,470958.281358,686.264003,0.806169,train,RandForest Estand IV


In [81]:
best_model12, result12 = rg_est.grid_s_decision(param5, "GradientBoosting Estand III")

In [82]:
result12

,MAE,MSE,RMSE,R2,set,modelo
0,574.268742,542678.376168,736.667073,0.772049,test,GradientBoosting Estand III
1,101.025112,22032.269530,148.432710,0.990932,train,GradientBoosting Estand III


In [83]:
best_model13, result13 = rg_sinest.grid_s_decision(param5, "GradientBoosting SinEstand III")

In [84]:
result13

,MAE,MSE,RMSE,R2,set,modelo
0,556.594613,527987.445026,726.627446,0.778220,test,GradientBoosting SinEstand III
1,104.605843,23240.097701,152.447032,0.990435,train,GradientBoosting SinEstand III


In [85]:
param7 = {"max_depth": [18,20], 
        "max_features": [4],
        "min_samples_split": [8, 16],
        "min_samples_leaf": [8]}

In [86]:
best_model14, result14 = rg_est.grid_s_decision(param7, "GradientBoosting Estand IV")

In [87]:
best_model14

GradientBoostingRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                          min_samples_split=16)

In [88]:
result14

,MAE,MSE,RMSE,R2,set,modelo
0,565.500705,572840.667153,756.862383,0.75938,test,GradientBoosting Estand IV
1,102.034904,22135.910308,148.781418,0.99089,train,GradientBoosting Estand IV
